In [8]:
!ls /keyakizaka_mining/model/facenet

20180402-114759  20180408-102900


In [9]:
!ls /keyakizaka_mining/data/faceimages

hamakishihiyori   katoushiho	      odanana		 takamotoayahana
haradaaoi	  kawatayouna	      ozekirika		 takaseaina
higashimuramei	  kobayashiyui	      saitoufuyuyuuhana  tannamameisato
hirateyurina	  koikeminami	      saitoukyouko	 tomitasuzuhana
inokuchimakotoo   kosakanao	      sasakibirei	 tsuchinamamizuho
ishimorinijihana  matsudakouhana      sasakikumi	 watanabemiho
kageyamayuuka	  miyataaihou	      satoushishoku	 watanabenashika
kakizakimemi	  moriyaakane	      shiosarina	 watanaberisa
kamimurarina	  nagahamaneru	      sugaiyuka
kanamurabiku	  nagasawana???kaori  suzuhonbiyu


In [10]:
# path

# save_dir = '/keyakizaka_mining/data/faceimages'
save_dir = '/keyakizaka_mining/data/faceimages_pad0'
#save_feature_dir = '/keyakizaka_mining/data/feature/faceimage'
save_feature_dir = '/keyakizaka_mining/data/feature/faceimage_pad0'
tool_dir = '/keyakizaka_mining/tools'
FACE_MEDEL_PATH = '/keyakizaka_mining/model/facenet/20180408-102900/20180408-102900.pb'

In [11]:
import os, sys
import numpy as np
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
%matplotlib inline

import random

import cloudpickle

def load_from_pkl( fpath ):
    frb = open(fpath, 'rb')
    obj = cloudpickle.loads(frb.read())
    return obj

def save_as_pkl( obj, fpath ):
    fwb = open( fpath, 'wb')
    tmp = fwb.write(cloudpickle.dumps(obj))
    return

In [12]:
# https://tech.unifa-e.com/entry/2018/09/20/183742
# > git clone https://github.com/davidsandberg/facenet.git

import tensorflow as tf

# add path for facenet model
sys.path.append(tool_dir)
from facenet.src import facenet

class FaceEmbedding(object):

    def __init__(self, model_path):
        facenet.load_model(model_path)
        
        self.input_image_size = 160
        self.sess = tf.Session()
        self.images_placeholder = tf.get_default_graph().get_tensor_by_name("input:0")
        self.embeddings = tf.get_default_graph().get_tensor_by_name("embeddings:0")
        self.phase_train_placeholder = tf.get_default_graph().get_tensor_by_name("phase_train:0")
        self.embedding_size = self.embeddings.get_shape()[1]
        
    def __del__(self):
        self.sess.close()
        
    def load_image(self, image_path, width, height, mode):
        image = Image.open(image_path)
        image = image.resize([width, height], Image.BILINEAR)
        return np.array(image.convert(mode))
        
    def face_embeddings(self, image_path):
        image = self.load_image(image_path, self.input_image_size, self.input_image_size, 'RGB')
        prewhitened = facenet.prewhiten(image)
        prewhitened = prewhitened.reshape(-1, prewhitened.shape[0], prewhitened.shape[1], prewhitened.shape[2])
        feed_dict = { self.images_placeholder: prewhitened, self.phase_train_placeholder: False }
        embeddings = self.sess.run(self.embeddings, feed_dict=feed_dict)
        return embeddings

In [13]:
from keras.preprocessing import image
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.models import Model
import numpy as np

model = VGG16(weights='imagenet', include_top=True)
# model.summary()

def vgg_fc1_feature( img_path ):
    img = image.load_img(img_path, target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    features = model.predict(x)
    model_extractfeatures = Model(inputs=model.input, outputs=model.get_layer('fc1').output)
    fc1_features = model_extractfeatures.predict(x)
    return fc1_features

def vgg_fc2_feature( img_path ):
    img = image.load_img(img_path, target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    features = model.predict(x)
    model_extractfeatures = Model(inputs=model.input, outputs=model.get_layer('fc2').output)
    fc2_features = model_extractfeatures.predict(x)
    return fc2_features

In [14]:
# save facenet features

# load model
face_embedding = FaceEmbedding(FACE_MEDEL_PATH)

# extract facenet features
dirs = os.listdir( save_dir )
for author in dirs:
    feature_dir = '%s/facenet' % (save_feature_dir)
    if not os.path.exists( feature_dir ): os.mkdir( feature_dir )
    # if os.path.exists( '%s/X_facenet_%s.pkl' % (feature_dir, author) ): continue
    
    # make resize_dir
    rdir = '%s_resize/%s' % (save_dir,author)
    if not os.path.exists( rdir ): os.mkdir( rdir )
    
    print( author, "Extract features..." )
    
    X = []
    y = []
    image_list  = []
    image_list_resize = []
    images = os.listdir( '%s/%s'% (save_dir,author) )
    for idx, imname in enumerate( images ):
        if idx % 100 == 0: print( idx )
        img_path = '%s/%s/%s'% (save_dir,author,imname)
        im = Image.open( img_path )
        
        if im.size[0] < 200: continue
        
        img_path_resize = '%s_resize/%s/resize160_%s'% (save_dir,author,imname)
        img_resize = im.resize((160, 160))
        img_resize.save(img_path_resize)
    
        feature = face_embedding.face_embeddings( img_path )[0]
        
        X.append( feature.flatten() )
        y.append( author )
        image_list.append( img_path )
        image_list_resize.append( img_path_resize )
    
    # save feature
    X = np.array(X)
    save_as_pkl( X, '%s/X_facenet_%s.pkl' % (feature_dir, author) )
    save_as_pkl( y, '%s/y_facenet_%s.pkl' % (feature_dir, author) )
    save_as_pkl( image_list, '%s/imagelist_facenet_%s.pkl' % (feature_dir, author) )
    save_as_pkl( image_list_resize, '%s/resize160_imagelist_facenet_%s.pkl' % (feature_dir, author) )

Model filename: /keyakizaka_mining/model/facenet/20180408-102900/20180408-102900.pb
takaseaina Extract features...
0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
tannamameisato Extract features...
0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
nagahamaneru Extract features...
0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
sugaiyuka Extract features...
0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
odanana Extract features...
0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
kawatayouna Extract features...
0
100
200
300
400
500
600
700
matsudakouhana Extract features...
0


In [15]:
# save vgg16_fc2

dirs = os.listdir( save_dir )
for author in dirs:
    feature_dir = '%s/vgg16_fc2' % (save_feature_dir)
    if not os.path.exists( feature_dir ): os.mkdir( feature_dir )
    # if os.path.exists( '%s/X_facenet_%s.pkl' % (feature_dir, author) ): continue
    
    print( author, "Extract features..." )

    # make resize_dir
    rdir = '%s_resize/%s' % (save_dir,author)
    if not os.path.exists( rdir ): os.mkdir( rdir )
    
    X = []
    y = []
    image_list  = []
    image_list_resize = []
    images = os.listdir( '%s/%s'% (save_dir,author) )
    for idx, imname in enumerate( images ):
        if idx % 100 == 0: print( idx )
        img_path = '%s/%s/%s'% (save_dir,author,imname)
        im = Image.open( img_path )
        
        if im.size[0] < 200: continue
        
        img_path_resize = '%s_resize/%s/resize160_%s'% (save_dir,author,imname)
        img_resize = im.resize((160, 160))
        img_resize.save(img_path_resize)
        
        feature = vgg_fc2_feature( img_path )
        
        X.append( feature.flatten() )
        y.append( author )
        image_list.append( img_path )
        image_list_resize.append( img_path_resize )
    
    # save feature
    X = np.array(X)
    save_as_pkl( X, '%s/X_vgg16_fc2_%s.pkl' % (feature_dir, author) )
    save_as_pkl( y, '%s/y_vgg16_fc2_%s.pkl' % (feature_dir, author) )
    save_as_pkl( image_list, '%s/imagelist_vgg16_fc2_%s.pkl' % (feature_dir, author) )
    save_as_pkl( image_list_resize, '%s/resize160_imagelist_vgg16_fc2_%s.pkl' % (feature_dir, author) )

takaseaina Extract features...
0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
tannamameisato Extract features...
0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
nagahamaneru Extract features...
0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
sugaiyuka Extract features...
0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
odanana Extract features...
0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
kawatayouna Extract features...
0
100
200
300
400
500
600
700
matsudakouhana Extract features...
0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
190